In [2]:
from cvzone import HandTrackingModule
import cv2
import numpy as np

def exponential_smoothing(current_point, prev_point, alpha=0.2):
    if prev_point is None:
        return current_point
    else:
        return tuple(map(int, np.multiply(current_point, alpha) + np.multiply(prev_point, 1 - alpha)))

def detect_hands():
    cap = cv2.VideoCapture(0)
    screen_width, screen_height = 1920, 1080
    cap.set(3, screen_width)
    cap.set(4, screen_height)

    if not cap.isOpened():
        print("Error: Could not open the camera.")
        return

    hand_detector = HandTrackingModule.HandDetector(detectionCon=0.8, maxHands=1)

    small_corner_width, small_corner_height = 320, 240
    cv2.namedWindow('Canvas with Small Video Corner', cv2.WINDOW_NORMAL)
    cv2.resizeWindow('Canvas with Small Video Corner', screen_width, screen_height)

    canvas = np.zeros((screen_height, screen_width, 3), dtype=np.uint8)
    drawn_paths = []

    drawing = False
    prev_smoothed_point = None

    while True:
        ret, frame = cap.read()

        if not ret:
            print("Error: Failed to capture frame.")
            break

        frame = cv2.flip(frame, 1)
        canvas_resized = cv2.resize(canvas, (frame.shape[1], frame.shape[0]))

        hands, _ = hand_detector.findHands(frame)

        if hands:
            hand = hands[0]
            index_finger_tip = hand['lmList'][8][:2]
            middle_finger_tip = hand['lmList'][12][:2]
            thumb_tip = hand['lmList'][4][:2]

            if np.linalg.norm(np.array(index_finger_tip) - np.array(thumb_tip)) < 40:
                drawn_paths = []  
                drawing = False
                prev_smoothed_point = None

            fingers_distance = np.linalg.norm(np.array(index_finger_tip) - np.array(middle_finger_tip))
            if fingers_distance < 40:
                drawing = False
                prev_smoothed_point = None
            else:
                drawing = True

            # Check if the hand is in a fist position based on thumb and finger positions
            if (
                thumb_tip[1] < index_finger_tip[1] < middle_finger_tip[1]
                and thumb_tip[0] > index_finger_tip[0] > middle_finger_tip[0]
            ):
                print("Fist is closed. Exiting the program.")
                break

            if drawing:
                smoothed_point = exponential_smoothing(index_finger_tip, prev_smoothed_point)
                if prev_smoothed_point:
                    cv2.line(canvas_resized, prev_smoothed_point, smoothed_point, (255, 255, 255), 5)
                    drawn_paths.append((prev_smoothed_point, smoothed_point))
                prev_smoothed_point = smoothed_point

        canvas_with_small_video = np.copy(canvas_resized)
        small_corner_frame = cv2.resize(frame, (small_corner_width, small_corner_height))
        canvas_with_small_video[10:10 + small_corner_height, 10:10 + small_corner_width] = small_corner_frame

        for path in drawn_paths:
            cv2.line(canvas_with_small_video, path[0], path[1], (255, 255, 255), 5)

        if drawing:
            cv2.circle(canvas_with_small_video, smoothed_point, 15, (255, 255, 255), -1)

        cv2.imshow('Canvas with Small Video Corner', canvas_with_small_video)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    detect_hands()

Fist is closed. Exiting the program.
